In [ ]:
using Pkg

# Pkg.add("DataFrames")
# Pkg.add("CSV")

using DataFrames, CSV

In [ ]:
# File from DUO: gemiddelde-afstand-tussen-woonadres-leerling-en-schoolvestiging-2020-2021

df = CSV.read("data.csv", DataFrame, delim=";", decimal=',');

In [ ]:
size(unique(df, :GEMEENTENUMMER))

In [ ]:
gdf = groupby(df, :GEMEENTENUMMER)

In [ ]:
using Statistics

distances = combine(gdf, :AFSTAND => mean)

In [ ]:
using GeoDataFrames; const GDF=GeoDataFrames

In [ ]:
# https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/wijk-en-buurtkaart-2020
gemeente = GDF.read("WijkBuurtkaart_2020_v2/gemeente_2020_v2.shp")

In [ ]:
# Change the Gemeente code into a number.
chop("GM0002", head = 2, tail = 0)

In [ ]:
transform!(gemeente, :GM_CODE => x -> chop.(x, head = 2, tail = 0));

In [ ]:
gemeente[!,:GM_CODE_function] = parse.(Int64, gemeente[:,:GM_CODE_function]);

In [ ]:
data = rightjoin(distances, gemeente, on = on = :GEMEENTENUMMER => :GM_CODE_function)

In [ ]:
using Plots

In [ ]:
row_filter = data.H2O .== "NEE";

In [ ]:
municipality_data = data[row_filter, :];

In [ ]:
municipality_shape = data.geom[row_filter, :];

In [ ]:
using LinearAlgebra

values_ = municipality_data[:, :AFSTAND_mean]
normalized_values = normalize(values_);

In [ ]:
using Plots

colors = Array([cgrad(:heat)[value] for value in normalized_values])

# plot thematic map
p = plot(dpi = 300, size=(500, 600), axis=false, ticks=false)
for i = 1:nrow(municipality_data)
    plot!(municipality_shape[i], color=colors[i])
end
plot!(plot_title="Average distance from student home to primary school", plot_titlefontsize=10)

In [ ]:
png(p, "distance.png")